- [1. Importing Functions and Packages](#1)
    - [Max Drawdown and Calmer Ratio

## 1. Importing Functions and Packages <a id='1'></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
def kelly(decimal_odds, P, fractional):
    # shrinkage is the percent we want to shrink bets to account for uncertanity
    B = decimal_odds - 1
    Q = 1 - P
    solution = (B*P - Q) / B
    solution = solution * fractional
    return solution

def backtest(df,initial, shrinkage, threshold):
    df = df.copy()
    df = df[df['Sportsbooks_Odds'] > threshold]
    #df = df[df['Sportsbooks_Odds'] < 2.00]
    df.loc[:, 'Kelly'] = kelly(df['Sportsbooks_Odds'], df['Probabilities'], shrinkage)
    df = df[df['Kelly'] > 0].reset_index(drop=True)
    df = df.sort_values(by='Date')
    balance = initial
    pnl = []
    track = []
    returns_list = [] 
    
    for index,row in df.iterrows():
        bet_amount = (row['Kelly']) * initial
        if row['Predictions'] == row['Actual']:
            profit = bet_amount * (row['Sportsbooks_Odds'] - 1)
            balance += profit
            pnl.append(profit)
            track.append(balance)
        else:
            balance -= bet_amount
            pnl.append(-bet_amount)
            track.append(balance)
        if index > 0:
            returns_list.append((balance / initial - 1) * 100)
            
    returns_std = np.std(returns_list)
    avg_return = np.mean(returns_list)
    sharpe_ratio = (avg_return / returns_std)
    
    balance = round(balance, 2)
    periods = len(df)
    returns = pow((balance / initial), (1/periods)) - 1
    returns = round(returns* 100, 4) 
    final_return = ((balance / initial) - 1) * 100
    
    plt.figure(figsize = (16,7))
    plt.plot(df['Date'], pnl, c='b')
    plt.xlabel('Time(d)', size=17)
    plt.ylabel('Profit and Loss($)', size=17)
    plt.title('PnL Over Time')
    
    plt.show()  # Show the first plot
    
    plt.figure(figsize = (16,7))
    plt.plot(df['Date'], track, c='b')
    plt.xlabel('Time(d)', size=17)
    plt.ylabel('Bankroll($)', size=17)
    plt.title('Bankroll Over Time')
    
    plt.show()
    
    print('\n')
    print(f'The position was held for 2 months, traded {periods} times, and produced a return of {final_return:.4f}%.')
    print(f'The growth rate on each bet was {returns:.4f}%.')
    print(f'The final balance is ${balance}.')
    print(f'The average of returns is {avg_return:.4f}%.')
    print(f'The standard deviation of returns is {returns_std:.4f}%.')
    print('\n')
    #print(track)
    print(f'The Sharpe Ratio of the strategy is {sharpe_ratio:.4f}.')


#### Max Drawdown and Calmer Ratio <a id='1_1'></a>

In [6]:
def calculate_max_drawdown(series):
    """
    Calculate drawdown from a time series of equity values.
    """
    max_dd = 0
    peak = series[0]
    drawdowns = []
    
    for val in series:
        if val > peak:
            peak = val
        dd = (peak - val) / peak
        drawdowns.append(dd)
        max_dd = max(max_dd, dd)
    
    return max_dd

def backtest_for_calmar(df, initial, shrinkage, threshold):
    """
    Backtest function to calculate the Calmar ratio.
    
    Parameters:
        df (DataFrame): DataFrame containing the backtest data.
        initial (float): Initial balance.
        shrinkage (float): Shrinkage factor for Kelly criterion.
        threshold (float): Minimum threshold for placing a bet.
        
    Returns:
        calmar_ratio (float): Calmar ratio.
    """
    df = df.copy()
    df = df[df['Sportsbooks_Odds'] > threshold]
    df.loc[:, 'Kelly'] = kelly(df['Sportsbooks_Odds'], df['Probabilities'], shrinkage)
    
    df = df[df['Kelly'] > 0].reset_index(drop=True)
    df = df.sort_values(by='Date')
    balance = initial
    pnl = []
    track = []
    returns_list = [] 
    
    for index, row in df.iterrows():
        bet_amount = row['Kelly'] * initial
        if row['Predictions'] == row['Actual']:
            profit = bet_amount * (row['Sportsbooks_Odds'] - 1)
            balance += profit
            pnl.append(profit)
            track.append(balance)
        else:
            balance -= bet_amount
            pnl.append(-bet_amount)
            track.append(balance)
        if index > 0:
            returns_list.append((balance / initial - 1) * 100)
    
    max_drawdown = calculate_max_drawdown(track)
    
    avg_return = np.mean(returns_list)
    
    if max_drawdown == 0:
        calmar_ratio = np.inf  # To handle division by zero
    else:
        calmar_ratio = avg_return / max_drawdown
    
    return calmar_ratio

#### Sortino Backtest <a id='1_2'></a>

In [7]:
def backtest_for_sortino(df, initial, shrinkage, threshold):
    df = df.copy()
    df = df[df['Sportsbooks_Odds'] > threshold]
    df.loc[:, 'Kelly'] = kelly(df['Sportsbooks_Odds'], df['Probabilities'], shrinkage)
    
    df = df[df['Kelly'] > 0].reset_index(drop=True)
    df = df.sort_values(by='Date')
    balance = initial
    pnl = []
    track = []
    returns_list = [] 
    
    for index, row in df.iterrows():
        bet_amount = row['Kelly'] * initial
        if row['Predictions'] == row['Actual']:
            profit = bet_amount * (row['Sportsbooks_Odds'] - 1)
            balance += profit
            pnl.append(profit)
            track.append(balance)
        else:
            balance -= bet_amount
            pnl.append(-bet_amount)
            track.append(balance)
        if index > 0:
            returns_list.append((balance / initial - 1) * 100)
    
    downside_returns = [ret for ret in returns_list if ret < 0]
    downside_deviation = np.std(downside_returns)
    
    if downside_deviation == 0:
        sortino_ratio = np.inf  # To handle division by zero
    else:
        average_return = np.mean(returns_list)
        sortino_ratio = average_return / downside_deviation
    
    return sortino_ratio